In [58]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to /Users/secilsair/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/secilsair/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [60]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [62]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [64]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

471 documents
46 classes ['About_Delizeroo', 'Fees_on_Delizeroo', 'Inviting_friends', 'about_my_order', 'admission', 'cancel_order', 'canteen', 'college intake', 'committee', 'computerhod', 'course', 'creator', 'delizeroo_editions', 'document', 'event', 'extchod', 'facilities', 'fees', 'floors', 'goodbye', 'greeting', 'hod', 'hostel', 'hours', 'infrastructure', 'ithod', 'library', 'location', 'menu', 'name', 'number', 'order_information', 'placement', 'principal', 'ragging', 'random', 'salutaion', 'scholarship', 'sem', 'sports', 'swear', 'syllabus', 'task', 'thanks', 'uniform', 'vacation']
304 unique lemmatized words ["'m", "'s", '(', ')', ',', '.', 'a', 'about', 'ac', 'accept', 'active', 'activity', 'add', 'address', 'admision', 'admission', 'against', 'ai/ml', 'allotment', 'am', 'an', 'and', 'antiragging', 'any', 'anyone', 'are', 'around', 'arrives', 'as', 'asshole', 'at', 'attend', 'automobile', 'available', 'average', 'awesome', 'be', 'between', 'big', 'bitch', 'book', 'boy', 'bran

In [66]:
pickle.dump(words,open('unique_words.pkl','wb'))
pickle.dump(classes,open('classes1.pkl','wb'))

In [68]:
from sklearn.model_selection import train_test_split

# create training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)

# Özellikleri ve etiketleri 'training' listesinden ayırma
X = [item[0] for item in training]  # features
y = [item[1] for item in training]  # label


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)



# Numpy dizisine dönüştürme
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)


In [70]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(X_train[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]), activation='softmax'))

In [72]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.metrics import Precision, Recall
import tensorflow as tf

tf.random.set_seed(123)

class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = Precision()
        self.recall = Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return 2 * ((p * r) / (p + r + tf.keras.backend.epsilon()))

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()



sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, 
              metrics=['accuracy', Precision(), Recall()])

In [74]:
#hist = model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=4, verbose=1)
#model.save('my_model.h5', hist)

In [76]:
#fitting and saving the model
np.random.seed(123)
import random as python_random
python_random.seed(123)
hist = model.fit(np.array(X_train), np.array(y_train), epochs=200, batch_size=6, verbose=1)
model.save('my_model.h5', hist)

Epoch 1/200
63/63 [==============================] - 0s 402us/step - loss: 3.8170 - accuracy: 0.0186 - precision_1: 0.0000e+00 - recall_1: 0.0000e+00
Epoch 2/200
63/63 [==============================] - 0s 343us/step - loss: 3.6967 - accuracy: 0.0771 - precision_1: 0.0000e+00 - recall_1: 0.0000e+00
Epoch 3/200
63/63 [==============================] - 0s 331us/step - loss: 3.5422 - accuracy: 0.1170 - precision_1: 0.0000e+00 - recall_1: 0.0000e+00
Epoch 4/200
63/63 [==============================] - 0s 336us/step - loss: 3.4077 - accuracy: 0.1596 - precision_1: 0.0000e+00 - recall_1: 0.0000e+00
Epoch 5/200
63/63 [==============================] - 0s 339us/step - loss: 3.1765 - accuracy: 0.2101 - precision_1: 0.8333 - recall_1: 0.0133
Epoch 6/200
63/63 [==============================] - 0s 342us/step - loss: 2.8828 - accuracy: 0.2819 - precision_1: 0.9429 - recall_1: 0.0878  
Epoch 7/200
63/63 [==============================] - 0s 339us/step - loss: 2.7004 - accuracy: 0.3032 - precision_1

In [77]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
from keras.models import load_model
import json
import random

In [78]:
from tensorflow.keras.models import load_model
import json
import pickle

model = load_model('my_model.h5', custom_objects={'F1Score': F1Score})
intents = json.loads(open('intents.json').read())
words = pickle.load(open('unique_words.pkl', 'rb'))
classes = pickle.load(open('classes1.pkl', 'rb'))


In [79]:
accuracy, precision, recall, f1_score = model.evaluate(X_test, y_test, verbose=0)

print(f"Doğruluk (Accuracy): {accuracy}")
print(f"Kesinlik (Precision): {precision}")
print(f"Duyarlılık (Recall): {recall}")
print(f"F1 Skoru: {f1_score}")


Doğruluk (Accuracy): 0.9378851652145386
Kesinlik (Precision): 0.8105263113975525
Duyarlılık (Recall): 0.8690476417541504
F1 Skoru: 0.7684210538864136


In [84]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag'] == tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res





In [ ]:
print("Welcome to Delizeroo! How can I help you?")
while True:
    msg = input("Customer: ")
    if msg == "quit":
        break
    response = chatbot_response(msg)
    print("Chatbot:", response)

Welcome to Delizeroo! How can I help you?


Customer:  Hello


1/1 [==============================] - 0s 48ms/step
Chatbot: Good to see you again!


Customer:  How does it work?


1/1 [==============================] - 0s 15ms/step
Chatbot: You can order either on the website or by using the Deliveroo app, available on iOS and Android. Simply add your postcode to find all the great restaurants delivering in your area, choose your food and place your order


...

1/1 [==============================] - 0s 15ms/step
Chatbot: Maintaining decency would be appreciated


Customer:  My order's late. Where is it?", "I would like to receive information about my order


1/1 [==============================] - 0s 16ms/step
Chatbot: Please provide full name and order number


Customer:  secil, 234


1/1 [==============================] - 0s 8ms/step
Chatbot: Talk to you later
